# Predicting Particulate Matter (PM2.5) Concentrations in the Air of China

# Implementation

In [7]:
# Import libraries necessary for this project
import csv
import numpy as np
import pandas as pd
from IPython.display import display # Allows the use of display() for DataFrames
from time import time
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# Dara preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from math import sqrt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
# Import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit


In [8]:
# Load Guangzhou dataset
Guangzhou = pd.read_csv('dataset_Guangzhou_clean.csv')
Guangzhou = Guangzhou.drop('Unnamed: 0', axis = 1)

# Load Beijing dataset
Beijing = pd.read_csv('dataset_Beijing_clean.csv')
Beijing = Beijing.drop('Unnamed: 0', axis = 1)

# Load Chengdu dataset
Chengdu = pd.read_csv('dataset_Chengdu_clean.csv')
Chengdu = Chengdu.drop('Unnamed: 0', axis = 1)

# Load Shanghai dataset
Shanghai = pd.read_csv('dataset_Shanghai_clean.csv')
Shanghai = Shanghai.drop('Unnamed: 0', axis = 1)

# Load Shenyang dataset
Shenyang = pd.read_csv('dataset_Shenyang_clean.csv')
Shenyang = Shenyang.drop('Unnamed: 0', axis = 1)

dataset = Beijing.append(Chengdu, ignore_index=True, sort=True)
dataset = dataset.append(Shanghai, ignore_index=True, sort=True)
dataset = dataset.append(Shenyang, ignore_index=True, sort=True)
dataset = dataset.append(Guangzhou, ignore_index=True, sort=True)



# Total number of records
n_records = len(dataset["PM_US Post"])
print("Number of records for all Chines cities: ", n_records)
print("*********************")
display(dataset.info())
display(dataset.describe())




Number of records for all Chines cities:  117200
*********************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117200 entries, 0 to 117199
Data columns (total 23 columns):
DEWP             117200 non-null float64
HUMI             117200 non-null float64
PM_US Post       117200 non-null float64
PRES             117200 non-null float64
TEMP             117200 non-null float64
cbwd_NE          117200 non-null int64
cbwd_NW          117200 non-null int64
cbwd_SE          117200 non-null int64
cbwd_SW          117200 non-null int64
cbwd_cv          117200 non-null int64
day_cos          117200 non-null float64
day_sin          117200 non-null float64
hour_cos         117200 non-null float64
hour_sin         117200 non-null float64
month_cos        117200 non-null float64
month_sin        117200 non-null float64
new_wind         117200 non-null float64
precipitation    117200 non-null float64
season_1         117200 non-null int64
season_2         117200 non-null int64
season_3    

None

,DEWP,HUMI,PM_US Post,PRES,TEMP,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW,cbwd_cv,...,hour_sin,month_cos,month_sin,new_wind,precipitation,season_1,season_2,season_3,season_4,year
count,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,...,117200.000000,1.172000e+05,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000,117200.000000
mean,8.989047,66.604351,71.029445,1013.568459,16.191724,0.221468,0.246937,0.222082,0.143567,0.165947,...,-0.002445,6.348879e-03,-0.009274,2.757414,0.124447,0.248857,0.242585,0.255222,0.253336,2014.011314
std,12.318217,22.354439,65.563051,9.906103,10.734711,0.415236,0.431232,0.415648,0.350651,0.372035,...,0.707228,7.050057e-01,0.709119,1.943324,1.117760,0.432352,0.428648,0.435988,0.434924,0.809783
min,-40.000000,2.000000,1.000000,975.000000,-25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000e+00,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000
25%,1.000000,51.560000,29.000000,1006.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.707107,-5.000000e-01,-0.866025,1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000
50%,12.000000,70.360000,52.000000,1013.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.836970e-16,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2014.000000
75%,19.000000,86.000000,90.000000,1021.000000,24.400000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.707107,8.660254e-01,0.500000,4.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2015.000000
max,28.000000,100.000000,932.000000,1046.000000,42.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000e+00,1.000000,20.120000,48.600000,1.000000,1.000000,1.000000,1.000000,2015.000000


In [9]:
# Machine learning algorithms decleration
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

LR = LinearRegression()
RF = RandomForestRegressor(n_estimators = 96)
ANN = MLPRegressor(hidden_layer_sizes= (128, 256))
SVR = SVR(kernel='rbf')
#SVR_tuned = SVR(kernel='rbf', C = 707, epsilon = 4)

MLs = {'LR' : LR, 'RF': RF, 'ANN' : ANN, 'SVR' : SVR}
#MLs = {'LR' : LR}


In [10]:


def apply_L5(cityTrainName, dataset, cityTestName, MLname, estimator, f_out, Un_needed_columns):
    # Construct the pipeline with a standard scaler and a small neural network
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append((MLname, estimator))
    model = Pipeline(estimators)

    # Split the data into features and target label
    #Un_needed_columns = ['PM_US Post', 'precipitation', 'day_cos', 'day_sin']
    # Split the data into features and target label
    Features = dataset.drop(Un_needed_columns, axis = 1)
    Target = dataset['PM_US Post']

    # Saving feature names for later use
    features_list = list(Features.columns)

    display(Features.head(n=2))
    display(Target.head(n=2))
    
    #Shuffle and Split Data
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(Features, Target, test_size=0.20, random_state=0, shuffle=True)

    
    # Feature Scaling
    #Sc_X = StandardScaler()
    #X_train = Sc_X.fit_transform(X_train)
    #X_test = Sc_X.transform(X_test)
    

    print(estimators)

    print("**Train Split **")
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    R2 = r2_score(y_test, predict)
    MSE =  mean_squared_error(y_test,predict)
    MAE =  mean_absolute_error(y_test,predict)
    RMSE = sqrt(MSE)

    print("Train City : ", cityTrainName)
    print("Test City : ", cityTestName)
    print("MSE : ", MSE)
    print("MAE : ", MAE)
    print("R2 : ", R2)
    print("RMSE : ", RMSE)

    print("-----------------------------------------------")

    
    f_out.write(str(cityTrainName) + ",")
    f_out.write(str(cityTestName) + ",")
    f_out.write(str(MLname) + ",")
    f_out.write('TTS' + ",")
    f_out.write(str(abs(MAE)) + ",")
    f_out.write(str(abs(MSE)) + ",")
    f_out.write(str(RMSE) + ",")
    f_out.write(str(R2) + ",")
    f_out.write(str(len(features_list)) + ",")
    for feature in features_list:
        f_out.write(feature + "&")
    f_out.write("\n")




In [11]:


def applyCV(cityTrainName, dataset, cityTestName, MLname, estimator, f_out, Un_needed_columns):
    # Construct the pipeline with a standard scaler and a small neural network
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append((MLname, estimator))
    model = Pipeline(estimators)

    # Split the data into features and target label
    #Un_needed_columns = ['PM_US Post', 'precipitation', 'day_cos', 'day_sin']
    
    # Split the data into features and target label
    X = dataset.drop(Un_needed_columns, axis = 1)
    y = dataset['PM_US Post']
    
    # Saving feature names for later use
    features_list = list(X.columns)

    # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
    # to train the model, and the prediction score will be computed on the remaining 20%.
    # This process is repeated five times such that the training sets in each "fold"
    # are mutually orthogonal.
    
    K = 3
    kfold = KFold(n_splits=K,  shuffle=True)

    print(estimators)

    print("**cross_val_score + KFold **")

    results_R2 = cross_val_score(model, X, y, cv=kfold, scoring='r2')
    R2 = np.mean(results_R2)
    print('CV Scoring Result: r2 : mean=',np.mean(results_R2),'std=',np.std(results_R2))
    #print(results_R2) 
    print("**************")
    
    results_MAE = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error')
    MAE = np.mean(results_MAE)
    print('CV Scoring Result: MAE : mean=',np.mean(results_MAE),'std=',np.std(results_MAE))
    #print(results_MAE)  
    print("**************")

    results_MSE = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
    MSE = np.mean(results_MSE)
    print('CV Scoring Result: MSE : mean=',np.mean(results_MSE),'std=',np.std(results_MSE))
    #print(results_MSE) 
    
    RMSE = sqrt(abs(MSE))
    
    print("-----------------------------------------------")

    
    f_out.write(str(cityTrainName) + ",")
    f_out.write(str(cityTestName) + ",")
    f_out.write(str(MLname) + ",")
    f_out.write('CV(' + str(K) + "),")
    f_out.write(str(abs(MAE)) + ",")
    f_out.write(str(abs(MSE)) + ",")
    f_out.write(str(RMSE) + ",")
    f_out.write(str(R2) + ",")
    f_out.write(str(len(features_list)) + ",")
    for feature in features_list:
        f_out.write(feature + "&")
    f_out.write("\n")




In [12]:
with open("../China/China_Results/Paper_Results/Level_5_Balanced_Test_20_features.csv", 'w') as f_out:
    out_colnames = ['Train Site', 'Test Site', 'Algorithm', 'CV', 'MAE', 'MSE', 'RMSE', 'R^2', 'Features_Count', 'Features']        
    writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    writer.writeheader()
    
    Train_city_Name = "Beijing & Chengdu & Shanghai & Shenyang & Guangzhou"
    Test_city_Name = "Beijing & Chengdu & Shanghai & Shenyang & Guangzhou"
    
    for MLname, ML in MLs.items():
        print(Train_city_Name, " ********************** and *********************  ", Test_city_Name)
        Un_needed_columns = ['PM_US Post','day_cos', 'day_sin', ]
        apply_L5(Train_city_Name, dataset, Test_city_Name, MLname, ML, f_out, Un_needed_columns)
        applyCV(Train_city_Name, dataset, Test_city_Name, MLname, ML, f_out, Un_needed_columns)

        

Beijing & Chengdu & Shanghai & Shenyang & Guangzhou  ********************** and *********************   Beijing & Chengdu & Shanghai & Shenyang & Guangzhou


,DEWP,HUMI,PRES,TEMP,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW,cbwd_cv,hour_cos,hour_sin,month_cos,month_sin,new_wind,precipitation,season_1,season_2,season_3,season_4,year
0,-10.0,67.0,1018.0,-5.0,0,1,0,0,0,1.000000,0.000000,1.0,0.0,4.02,0.0,0,0,0,1,2013
1,-11.0,73.0,1017.0,-7.0,0,1,0,0,0,0.965926,0.258819,1.0,0.0,4.02,0.0,0,0,0,1,2013


0    31.0
1    32.0
Name: PM_US Post, dtype: float64

[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**Train Split **
Train City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
Test City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
MSE :  3269.1737233924378
MAE :  38.69939533526204
R2 :  0.2390504862425702
RMSE :  57.176688636125455
-----------------------------------------------
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**cross_val_score + KFold **


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

...

CV Scoring Result: r2 : mean= 0.2885361596587062 std= 0.001801561377853273
**************


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: MAE : mean= -33.093196367077105 std= 0.2519402488797445
**************


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: MSE : mean= -3057.434320572098 std= 49.721066256608474
-----------------------------------------------


In [13]:
with open("../China/China_Results/Paper_Results/Level_5_Balanced_Test_22_features.csv", 'w') as f_out:
    out_colnames = ['Train Site', 'Test Site', 'Algorithm', 'CV', 'MAE', 'MSE', 'RMSE', 'R^2', 'Features_Count', 'Features']        
    writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    writer.writeheader()
    
    Train_city_Name = "Beijing & Chengdu & Shanghai & Shenyang & Guangzhou"
    Test_city_Name = "Beijing & Chengdu & Shanghai & Shenyang & Guangzhou"
    
    for MLname, ML in MLs.items():
        print(Train_city_Name, " ********************** and *********************  ", Test_city_Name)
        Un_needed_columns = ['PM_US Post','day_cos', 'day_sin', 'precipitation']
        apply_L5(Train_city_Name, dataset, Test_city_Name, MLname, ML, f_out, Un_needed_columns)
        applyCV(Train_city_Name, dataset, Test_city_Name, MLname, ML, f_out, Un_needed_columns)

        

Beijing & Chengdu & Shanghai & Shenyang & Guangzhou  ********************** and *********************   Beijing & Chengdu & Shanghai & Shenyang & Guangzhou


,DEWP,HUMI,PRES,TEMP,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW,cbwd_cv,day_cos,...,hour_sin,month_cos,month_sin,new_wind,precipitation,season_1,season_2,season_3,season_4,year
0,-10.0,67.0,1018.0,-5.0,0,1,0,0,0,-0.222521,...,0.000000,1.0,0.0,4.02,0.0,0,0,0,1,2013
1,-11.0,73.0,1017.0,-7.0,0,1,0,0,0,-0.222521,...,0.258819,1.0,0.0,4.02,0.0,0,0,0,1,2013


0    31.0
1    32.0
Name: PM_US Post, dtype: float64

[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**Train Split **
Train City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
Test City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
MSE :  3268.3927187825984
MAE :  38.69712188355777
R2 :  0.23923227685034587
RMSE :  57.16985848139383
-----------------------------------------------
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**cross_val_score + KFold **


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

...

CV Scoring Result: r2 : mean= 0.28436111414054405 std= 0.0005893260031256294
**************


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: MAE : mean= -33.15166584940852 std= 0.17161952008791997
**************


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: MSE : mean= -3073.495706207343 std= 70.11814797270871
-----------------------------------------------
